# Appendix - Helper Function Collections

## Init

In [ ]:
import sys
from pathlib import Path
SRC_PATH = Path().resolve() / 'src'
sys.path.append(str(SRC_PATH))

from src.config import REPO_PATH, DATA_PATH, DATA_21_PATH

## 2021 Census Microdata Preparation

In [ ]:
import dask.dataframe as dd
import osmnx as ox

from shapely.ops import unary_union

def census_microdata_filtering():
    """extract records from the whole census microdata with selected areas (inner london) and variables"""
    origin_microdata_file = DATA_21_PATH / 'UKDA2021census.tab' # To be processed file path
    filtered_microdata_file = DATA_21_PATH / 'census_microdata_2021_inner_london.csv'   # Output file path

    # filter regions (including 13 boroughs in Inner London, referred to:
    # https://www.london.gov.uk/programmes-strategies/planning/london-plan/the-london-plan-2021-online/annex-2-inner-and-outer-london-boroughs)
    inner_london_borough_codes = [
        "E68000208", "E68000214", "E68000218", "E68000219", "E68000220",
        "E68000226", "E68000227", "E68000229", "E68000230", "E68000235",
        "E68000237", "E68000239", "E68000232"] 
    
    variable_rename_map = {
        "resident_id_m":                "resident_id",
        "approx_social_grade":          "approximated_social_grade",
        "economic_activity_status_15m": "economic_activity",
        "employment_status":            "employment_status",
        "english_proficiency_5a":       "english_proficiency",
        "ethnic_group_tb_20b":          "ethnic_group",
        "gltla22cd":                    "borough_code",
        "hh_size_7a":                   "household_size",
        "highest_qualification":        "highest_qualification",
        "legal_partnership_status_7a":  "marital_status",
        "number_of_cars_6a":            "number_of_cars_and_vans",
        "resident_age_18m":             "age",
        "sex":                          "sex",
    }


    ddf_all = dd.read_table(origin_microdata_file, dtype={'fm_iol22cd': 'object'})
    ddf_filtered = ddf_all[ddf_all["gltla22cd"].isin(inner_london_borough_codes)]   # filter specific areas
    ddf_filtered = ddf_filtered[list(variable_rename_map.keys())]                   # filter specific variables
    ddf_filtered = ddf_filtered.rename(columns=variable_rename_map)                 # rename variables
    df_filtered = ddf_filtered.compute()
    df_filtered.to_csv(filtered_microdata_file, index=False)
    
    print(f'Filtered data successfully saved to {filtered_microdata_file}!')

# Call the function
census_microdata_filtering()

## LAD Population Feature Distribution Compare

In [ ]:
from src.config import RES_STATIC, RES_SYN_POP_PATH
import pandas as pd
from src.get_marginal_distribution import marg_age_dist, marg_sex_dist,marg_ethnic_dist, marg_ecoact_dist, marg_car_dist, marg_leptnershp_dist
from src.validation import freq_table, convert_freq_dict_to_vector, get_js_distance
REF_label = {
    'age': marg_age_dist(space_level='LAD'),
    'sex': marg_sex_dist(space_level='LAD'),
    'ethnic_group': marg_ethnic_dist(space_level='LAD'),
    'economic_activity': marg_ecoact_dist(space_level='LAD'),
    'number_of_cars_and_vans': marg_car_dist(space_level='LAD'),
    'marital_status': marg_leptnershp_dist(space_level='LAD'),
}
CODE_map = pd.read_csv(RES_STATIC / 'Coding_Scheme_LMSOA_LAD.csv')
CODE_area = CODE_map['LAD_CODE'].unique()

check_instances = [
                   '1e3',
                #    '1e3', '2e3', '3e3', '4e3', '5e3', '6e3', '7e3', '8e3', '9e3',
                   '1e4', '2e4', '3e4', '4e4', '5e4', '6e4', '7e4', '8e4', '9e4', 
                   '1e5', '2e5', '3e5', '4e5', '5e5', '6e5', '7e5', '8e5', '9e5',
                   '1e6', '1.5e6', '2e6'] 
check_features = ['age', 'sex', 'ethnic_group', 'economic_activity', 'number_of_cars_and_vans', 'marital_status']

#### Age

In [ ]:
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['age'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

In [ ]:

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = ['0-15', '16-29', '30-49', '50+']
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Age Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Age Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

In [ ]:
REF_label.keys()

#### Sex

In [ ]:
#### Age
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['sex'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = ['Female', 'Male']
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Sex Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Sex Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

#### ethnic_group

In [ ]:
probs_df.columns

In [ ]:
#### Age
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['ethnic_group'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = probs_df.columns
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Ethnic Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Ethnic Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

#### economic_activity

In [ ]:
#### Age
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['economic_activity'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = probs_df.columns
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Ecoact Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Ecoact Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

#### number_of_cars_and_vans

In [ ]:
#### Age
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['number_of_cars_and_vans'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = probs_df.columns
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Cars Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Cars Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

#### marital_status

In [ ]:
#### Age
from scipy.spatial.distance import jensenshannon
import numpy as np

probs_df = REF_label['marital_status'][:14]  # In JS Matrix, each row represent one distribution
probs = probs_df.to_numpy() 

# Calculate Jensen-Shannon Distribution Matrix to get the proximity of the frequency distribution
n = len(probs)
js_matrix = np.zeros((n, n))

for i in range(n):
    for j in range(i+1, n):
        dist = round(jensenshannon(probs[i], probs[j]), 3)
        js_matrix[i, j] = dist
        js_matrix[j, i] = dist

# print("Jensen-Shannon Distance Matrix:")
# print(js_matrix)

# Visualization
import matplotlib.pyplot as plt
labels = probs_df.index

# left plot
x = probs_df.columns
fig1, ax1 = plt.subplots(figsize=(8, 4))
for col in labels:
    ax1.plot(x, probs_df.loc[col], label=col)

ax1.set_xlabel('Marital Group')
ax1.set_ylabel('Percentage')
ax1.set_title('Marital Group by Different Local Authority')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))
ax1.grid(True)
plt.tight_layout()
plt.show()

# right plot
import seaborn as sns
fig2, ax2 = plt.subplots(figsize=(10, 8))
sns.heatmap(js_matrix, xticklabels=labels, yticklabels=labels, cmap='coolwarm', annot=True, ax=ax2,
            vmin=0, vmax=0.18)
ax2.set_title('Jensen-Shannon Distance Heatmap')
plt.tight_layout()
plt.show()

## Other Collections

### Convert from xlsx to csv

In [ ]:
## Change all xlsx to csvs under same file

import pandas as pd
from pathlib import Path
from tqdm import tqdm

def convert_xlsx_to_csv(folder_path):
    """
    Convert all .xlsx files in the specified folder to .csv files,
    keeping the original file names (only changing the extension).
    
    Parameters:
        folder_path (str or Path): Path to the folder containing .xlsx files
    """
    folder = Path(folder_path)
    xlsx_files = list(folder.glob("*.xlsx"))

    for xlsx_file in tqdm(xlsx_files, desc="Converting Progress", total=len(xlsx_files)):
        try:
            # Read the Excel file (first sheet by default)
            df = pd.read_excel(xlsx_file)

            # Generate corresponding .csv file path
            csv_file = xlsx_file.with_suffix(".csv")

            # Save as CSV (without index column)
            df.to_csv(csv_file, index=False)
            print(f"Converted: {xlsx_file.name} to {csv_file.name}")
        except Exception as e:
            print(f"Failed to process {xlsx_file.name}: {e}")

convert_xlsx_to_csv(DATA_21_PATH / 'statistic-summary')

### Collect Encoding Scheme from Shape file

In [ ]:
## Collect Encoding Scheme of different levels of division
import geopandas as gpd
# Inner London Area shape file
ILA_shp_file = DATA_PATH / 'gis-files' / 'Inner_London_Boundary' / 'Inner_London_lsoa.shp'
ILA_gpd = gpd.read_file(ILA_shp_file)
ILA_gpd.to_crs(epsg='4326', inplace=True)
ILA_lmsoa_code = ILA_gpd[['lsoa21cd', 'msoa21cd', 'lad22cd', 'lad22nm']].copy()
ILA_lmsoa_code.rename(columns={'lsoa21cd':'LSOA_CODE', 'msoa21cd':'MSOA_CODE', 'lad22cd':'LAD_CODE', 'lad22nm':'LAD_NAME'}, inplace=True)
ILA_lmsoa_code.to_csv(DATA_PATH / 'gis-files' / 'Coding_Scheme_LMSOA_LAD.csv', index=False) # Save the coding scheme of different levels of administrative regions


### Download Open Street Map for Landuse

In [ ]:
# import osmnx as ox
# import geopandas as gpd

# place_name = "London, UK"

# tags = {"landuse": True}
# gdf = ox.features_from_place(place_name, tags)
# gdf = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])]
# print(f"Filtered to {len(gdf)} polygon features.")

# print(gdf[["landuse", "geometry"]].head())

# output_path = DATA_PATH / 'gis-files' / 'OSM_data' /"london_landuse.shp"
# gdf.to_file(output_path, driver="ESRI Shapefile")

# print(f"Save as Shapefile：{output_path}")


In [ ]:
import geopandas as gpd
from shapely.ops import unary_union
import osmnx as ox

london_wards_shpfile = DATA_PATH / 'gis-files' / 'Inner_London_Boundary' / 'inner_london_lsoa.shp'
gdf_london_wards = gpd.read_file(london_wards_shpfile)
gdf_london_wards.to_crs(epsg=4326, inplace=True)


london_polygon = unary_union(gdf_london_wards["geometry"])

In [ ]:
import geopandas as gpd
from shapely.ops import unary_union
import osmnx as ox
from osmnx.features import features_from_polygon
from pathlib import Path


def get_osm_data_london():
    london_wards_shpfile = DATA_PATH / 'gis-files' / 'Inner_London_Boundary' / 'inner_london_lsoa.shp'
    gdf_london_wards = gpd.read_file(london_wards_shpfile)
    gdf_london_wards.to_crs(epsg=4326, inplace=True)

    london_polygon = unary_union(gdf_london_wards.geometry)

    out_dir = DATA_PATH / 'gis-files' / 'OSM_data'
    out_dir.mkdir(parents=True, exist_ok=True)

    
    gdf_landuse = features_from_polygon(
        london_polygon,
        tags={"landuse": ["residential", "retail", "commercial"]}
    )
    if not gdf_landuse.empty:
        gdf_landuse = gdf_landuse[["name", "landuse", "geometry", "type"]]
    gdf_landuse.to_file(str(out_dir / 'landuse.geojson'), driver='GeoJSON')

    
    gdf_buildings = features_from_polygon(
        london_polygon,
        tags={"building": ["residential"]}
    )
    gdf_buildings.to_file(str(out_dir / 'buildings.geojson'), driver='GeoJSON')

    
    G = ox.graph_from_polygon(
        london_polygon,
        custom_filter="['highway'~'motorway|trunk|primary|secondary|tertiary|residential']"
    )
    ox.io.save_graph_geopackage(
        G,
        filepath=str(out_dir / 'road_network.gpkg'),
        encoding='utf-8',
        directed=False
    )

    print(f"Saved to {out_dir.resolve()}")


### Convert geojson to shp

In [ ]:
import geopandas as gpd

out_dir = DATA_PATH / 'gis-files' / 'OSM_data'

# read GeoJSON file
geojson_path = out_dir / 'landuse.geojson'
gdf = gpd.read_file(geojson_path)

print("Finish Conversion!")


In [ ]:
out_dir = DATA_PATH / 'gis-files' / 'OSM_data'
gdf = gpd.read_file(out_dir / 'landuse.geojson')
polygon_gdf = gdf[gdf.geometry.type.isin(['Polygon', 'MultiPolygon'])]
polygon_gdf = polygon_gdf.reset_index(drop=True)
polygon_gdf = polygon_gdf.set_crs(epsg=4326, allow_override=True)
print(polygon_gdf.head())

# Save Shapefile
shp_output_path = out_dir / 'shp' / 'landuse.shp'  
polygon_gdf.to_file(shp_output_path, driver='ESRI Shapefile')

# gdf.to_file("landuse_fixed.shp", driver="ESRI Shapefile")

### Convert gpkg to shp

In [ ]:
from src.config import RES_SYN_ORD_PATH, RES_STATIC
import geopandas as gpd

gdf = gpd.read_file(RES_SYN_ORD_PATH / "synthetic_order_2e6_42.gpkg", layer="syn_ord")  # 如果gpkg里有多个layer，需指定
gdf.to_file(RES_STATIC / 'Figures'/ "Display_order.shp", driver="ESRI Shapefile", encoding="utf-8")

## Validation Using Sampling Directly from Syn Ord

### Validation of Synthetic Orders - Order Feature

Validation Data Source: [Online shopping behavior in the United Kingdom (UK)](https://www.statista.com/study/22395/online-shopping-in-the-united-kingdom-statista-dossier/)

In [ ]:
import statistics as stat
import pandas as pd
from tqdm import tqdm

from src.config import RES_STATIC, RES_SYN_ORD_PATH

REF_cate = pd.read_excel(RES_STATIC/'verify_order_category.xlsx', sheet_name='data')    # 1 - third party source

check_features = ['Order_Category']


In [ ]:
sample_sizes = [
    1e6, 2e6, 
    1e5, 2e5, 3e5, 4e5, 5e5, 6e5, 7e5, 8e5, 9e5, 
    1e4, 2e4, 3e4, 4e4, 5e4, 6e4, 7e4, 8e4, 9e4, 
    1e3, 2e3, 3e3, 4e3, 5e3, 6e3, 7e3, 8e3, 9e3, 
    1e2, 2e2, 3e2, 4e2, 5e2, 6e2, 7e2, 8e2, 
    1e1, 2e1, 5e1, 8e1, 
    ]
sample_sizes = list(map(int, sample_sizes))
random_seeds = [i for i in range(20)] + [42]

In [ ]:
import statistics as stat 
import pandas as pd
from tqdm import tqdm

from src.validation import freq_table, convert_freq_dict_to_vector, get_js_distance

res = []

for seed in tqdm(random_seeds, desc=f'Checking Progress'):
    instance = '2e6'
    syn_ord_name = 'synthetic_order_' + instance + f'_{seed}.csv'
    syn_ord_df = pd.read_csv(RES_SYN_ORD_PATH / syn_ord_name)

    for sample_size in sample_sizes:
        syn_ord_sample = syn_ord_df.sample(n=sample_size, replace=False)

        check_res = {}
        check_res['pop_size'] = sample_size
        check_res['seed'] = seed

        for feature in check_features:
        
            samples_X = syn_ord_sample[feature]
            
            freq_X = freq_table(samples_X)
            freq_Y = REF_cate['Probability'].to_dict()

            # Build the common support set (union of values)
            support_1d = sorted(set(freq_X) | set(freq_Y))

            # Convert freq dicts to aligned vectors
            p_vec = convert_freq_dict_to_vector(freq_X, support_1d)
            q_vec = convert_freq_dict_to_vector(freq_Y, support_1d)

            # Compute JS distance
            js_dist_1d = get_js_distance(p_vec, q_vec)
            # print(js_dist_1d)

            # js_var =  stat.variance(js_values_per_area)     # variance
            check_res[feature] = js_dist_1d
        
        res.append(check_res)

order_verify_df = pd.DataFrame(res)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

plt.rcParams['font.family'] = 'Arial'  # Times New Roman
plt.rcParams['xtick.labelsize'] = 12

fig, ax = plt.subplots(figsize=(12, 6))

df = order_verify_df

# 1️⃣ category and width
categories = np.sort(df['pop_size'].unique())
positions = categories
widths = categories/3

# prepare data
data = [df.loc[df['pop_size']==cat, "Order_Category"] for cat in categories]

# 2️⃣ Main: violin plot
parts = ax.violinplot(
    dataset=data,
    positions=positions,
    widths=widths,
    showmeans=False,
    showmedians=False,
    showextrema=False
)

for pc in parts['bodies']:
    pc.set_facecolor('#4d8075')     # light green
    pc.set_edgecolor('black')
    pc.set_alpha(0.5)

# 3️⃣ Main: boxplot plot
medians = []
for i, cat_data in enumerate(data):
    bp = ax.boxplot(
        cat_data,
        positions=[positions[i]],
        widths=widths[i]*0.3,
        patch_artist=True,
        boxprops=dict(facecolor='#335f69', color='black', linewidth=1),  # dark green
        medianprops=dict(color='red', linewidth=2),
        whiskerprops=dict(color='black', linewidth=1),
        capprops=dict(color='black', linewidth=1)
    )
    # get median
    median_val = bp['medians'][0].get_ydata()[0]
    medians.append(median_val)

# 4️⃣ Connect medians with a line
ax.plot(positions, medians, color='red', linestyle=':', marker='o', label='Median Trend', alpha=0.5)

# 5️⃣ set log x axis
ax.set_xscale('log')
ax.set_ylim(0, 0.9)
ax.set_xlim(5, 3e6)

# 6️⃣ labels
ax.set_xlabel("Synthetic Scale (pop_size)", fontsize=14)
ax.set_ylabel("Jenson-Shannon Divergence", fontsize=14)
ax.set_title('Violin and Boxplot of JSD for Synthetic Order Category', fontsize=14)

# 7️⃣ Add grid
ax.grid(which='major', axis='both', linestyle=':', linewidth=0.8, alpha=0.7)

# 8️⃣ Add legend
legend_elements = [
    Patch(facecolor='#4d8075', edgecolor='black', alpha=0.5, label='Violin Plot'),
    Patch(facecolor='#335f69', edgecolor='black', label='Boxplot'),
    Line2D([0], [0], color='red', linestyle=':', lw=2, label='Median Trend')
]
ax.legend(handles=legend_elements, loc='upper right', fontsize=12)

plt.tight_layout()
# plt.savefig(RES_STATIC / 'Figures'/ "jsd-ord.png", dpi=600, bbox_inches='tight') 
# plt.savefig(RES_STATIC / 'Figures'/ "jsd-ord.pdf", bbox_inches='tight') 
# plt.savefig(RES_STATIC / 'Figures'/ "poster-jsd-ord.png", dpi=600, bbox_inches='tight') 
plt.show()